# Practice with simple small graph

In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [10]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.memory import MemorySaver

In [4]:
class AgentState(TypedDict):
    lnode: str
    pad: str
    count: Annotated[int, operator.add]

In [6]:
def node1(state: AgentState):
    print(f"node1, count:{state['count']}")
    return {"lnode": "node_1",
            "count": 1}

def node2(state: AgentState):
    print(f"node2, count:{state['count']}")
    return {"lnode": "node_2",
            "count": 1}

def should_continue(state: AgentState) -> bool:
    return state["count"] < 3

The graph should go N1 -> N2 -> N1 but break

In [11]:
builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)
builder.add_edge("Node1", "Node2")
builder.add_conditional_edges("Node2", should_continue, { True: "Node1", False: END})
builder.set_entry_point("Node1")

#memory = SqliteSaver.from_conn_string(":memory:")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [12]:
thread = {"configurable": {"thread_id": "1"}}

graph.invoke({"count": 0, "pad": "Hello"}, thread)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'pad': 'Hello', 'count': 4}